# Postprocessing

## Setup

In [ ]:
import pandas as pd
import json
import numpy as np 

# Display options to see more data in the notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

## Load the Dataset

In [15]:
# Load the specific file uploaded (fill in here)
file_path = '../dataset/french_speeches_2000-01-01_to_2010-12-31.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.read_json(file_path)

print(f"Initial shape: {df.shape}")
df.head(2)

Initial shape: (39992, 19)


,id,titre,url,domaine,prononciation,intervenants,auteur_moral,circonstance,type_emetteur,type_document,type_media,media,resume,thematiques,descripteurs,mise_en_ligne,mise_a_jour,source,texte
0,117000001.0,"Déclaration de M. Nicolas Sarkozy, Président d...",https://www.vie-publique.fr/discours/180848-de...,Président de la République,2010-12-31,"[{'nom': 'Nicolas Sarkozy', 'qualite': None, '...",[],"Voeux radiotélévisés aux Français, le 31 décem...",Président de la République,Déclaration,None,None,None,[Distinction],"[Voeux, Crise economique, Crise financière, Cr...",2019-07-05,2025-05-13,None,"Mes chers compatriotes,\nL'année 2010 s'achève..."
1,113000025.0,"Déclaration de M. Alain Juppé, ministre de la ...",https://www.vie-publique.fr/discours/180847-de...,Déclaration,2010-12-31,"[{'nom': 'Alain Juppé', 'qualite': None, 'qual...",[],Voeux aux personnels civils et militaires du m...,Gouvernement,Déclaration,None,None,None,[Politique de la défense],"[Politique de la défense, Réforme, Interventio...",2019-07-05,2025-05-13,"http://www.defense.gouv.fr, le 3 janvier 2011","En cette fin d'année 2010, quelques semaines a..."


## French key name to English key name

In [16]:
# Create the translation mapping
french_to_english = {
    "id": "id",
    "titre": "title",
    "url": "source_url",
    "domaine": "domain",
    "prononciation": "date_pronounced",
    "intervenants": "speakers",
    "nom": "speaker_name",
    "qualite": "speaker_role",
    "qualite_long": "speaker_role_extended",
    "auteur_moral": "corporate_author",
    "circonstance": "circumstance",
    "type_emetteur": "issuer_type",
    "type_document": "document_type",
    "thematiques": "themes",
    "descripteurs": "tags",
    "mise_en_ligne": "date_published",
    "mise_a_jour": "date_updated",
    "texte": "text",
    "source": "source"
}

## (Optional) Apply Key Translation

In [17]:
# 1. Rename DataFrame Columns
# Only rename columns that actually exist in the dataframe to avoid errors
existing_cols_map = {k: v for k, v in french_to_english.items() if k in df.columns}
df.rename(columns=existing_cols_map, inplace=True)

# 2. Rename keys INSIDE the 'speakers' (formerly 'intervenants') list structure
# We map the specific nested keys defined in your dictionary
nested_speaker_map = {
    "nom": "speaker_name",
    "qualite": "speaker_role",
    "qualite_long": "speaker_role_extended"
}

def translate_nested_structure(speaker_list):
    if not isinstance(speaker_list, list):
        return speaker_list
    
    new_list = []
    for speaker in speaker_list:
        new_speaker = {}
        for k, v in speaker.items():
            # If the key is in our map, use the English version, otherwise keep original
            new_key = nested_speaker_map.get(k, k) 
            new_speaker[new_key] = v
        new_list.append(new_speaker)
    return new_list

# Apply the nested translation
df['speakers'] = df['speakers'].apply(translate_nested_structure)

# Verify the change
print("Columns translated.")
print("Sample of nested 'speakers' structure:", df['speakers'].iloc[0])
df.head(2)

Columns translated.
Sample of nested 'speakers' structure: [{'speaker_name': 'Nicolas Sarkozy', 'speaker_role': None, 'speaker_role_extended': 'FRANCE. Président de la République'}]


,id,title,source_url,domain,date_pronounced,speakers,corporate_author,circumstance,issuer_type,document_type,type_media,media,resume,themes,tags,date_published,date_updated,source,text
0,117000001.0,"Déclaration de M. Nicolas Sarkozy, Président d...",https://www.vie-publique.fr/discours/180848-de...,Président de la République,2010-12-31,"[{'speaker_name': 'Nicolas Sarkozy', 'speaker_...",[],"Voeux radiotélévisés aux Français, le 31 décem...",Président de la République,Déclaration,None,None,None,[Distinction],"[Voeux, Crise economique, Crise financière, Cr...",2019-07-05,2025-05-13,None,"Mes chers compatriotes,\nL'année 2010 s'achève..."
1,113000025.0,"Déclaration de M. Alain Juppé, ministre de la ...",https://www.vie-publique.fr/discours/180847-de...,Déclaration,2010-12-31,"[{'speaker_name': 'Alain Juppé', 'speaker_role...",[],Voeux aux personnels civils et militaires du m...,Gouvernement,Déclaration,None,None,None,[Politique de la défense],"[Politique de la défense, Réforme, Interventio...",2019-07-05,2025-05-13,"http://www.defense.gouv.fr, le 3 janvier 2011","En cette fin d'année 2010, quelques semaines a..."


## Analyze Empty Data

In [18]:
# Calculate null counts and percentages
null_counts = df.isnull().sum()
null_pct = (df.isnull().sum() / len(df)) * 100

missing_data = pd.DataFrame({'Missing Count': null_counts, 'Missing %': null_pct})
missing_data = missing_data[missing_data['Missing Count'] > 0].sort_values('Missing %', ascending=False)

print("--- Missing Data Analysis ---")
display(missing_data)

--- Missing Data Analysis ---


,Missing Count,Missing %
resume,39963,99.927485
type_media,31639,79.113323
media,30968,77.435487
circumstance,17781,44.461392
source,13698,34.251850
issuer_type,3422,8.556711
document_type,33,0.082517
id,1,0.002501
domain,1,0.002501


## Drop Empty Columns

In [19]:
# Define a dropoff threshold
threshold_percent = 95
columns_to_drop = missing_data[missing_data['Missing %'] > threshold_percent].index.tolist()

print(f"Dropping the following columns (> {threshold_percent}% missing): {columns_to_drop}")
print(f"Shape before dropping: {df.shape}")
df_clean = df.drop(columns=columns_to_drop)

# Double check shape
print(f"New shape: {df_clean.shape}")

Dropping the following columns (> 95% missing): ['resume']
Shape before dropping: (39992, 19)
New shape: (39992, 18)


## Drop Empty Text Rows

In [20]:
display(df_clean.head(3))

,id,title,source_url,domain,date_pronounced,speakers,corporate_author,circumstance,issuer_type,document_type,type_media,media,themes,tags,date_published,date_updated,source,text
0,117000001.0,"Déclaration de M. Nicolas Sarkozy, Président d...",https://www.vie-publique.fr/discours/180848-de...,Président de la République,2010-12-31,"[{'speaker_name': 'Nicolas Sarkozy', 'speaker_...",[],"Voeux radiotélévisés aux Français, le 31 décem...",Président de la République,Déclaration,None,None,[Distinction],"[Voeux, Crise economique, Crise financière, Cr...",2019-07-05,2025-05-13,None,"Mes chers compatriotes,\nL'année 2010 s'achève..."
1,113000025.0,"Déclaration de M. Alain Juppé, ministre de la ...",https://www.vie-publique.fr/discours/180847-de...,Déclaration,2010-12-31,"[{'speaker_name': 'Alain Juppé', 'speaker_role...",[],Voeux aux personnels civils et militaires du m...,Gouvernement,Déclaration,None,None,[Politique de la défense],"[Politique de la défense, Réforme, Interventio...",2019-07-05,2025-05-13,"http://www.defense.gouv.fr, le 3 janvier 2011","En cette fin d'année 2010, quelques semaines a..."
2,112000004.0,"Communiqué de la Présidence de la République, ...",https://www.vie-publique.fr/discours/180837-co...,Communiqué,2010-12-30,[{'speaker_name': 'Présidence de la République...,[Présidence de la République],None,Présidence de la République,Communiqué,None,None,[Santé publique],"[Santé publique, Nomination, Haute autorité de...",2019-07-05,2025-05-13,None,Le Président de la République tient à saluer l...


In [21]:
# 1. Identify rows where text is null, empty, or just whitespace
# We use a mask so we don't modify the dataframe yet
empty_mask = df_clean['text'].isnull() | (df_clean['text'].astype(str).str.strip() == "")

# 2. Grab the bad rows
bad_rows = df_clean[empty_mask]

if len(bad_rows) > 0:
    print(f"Found {len(bad_rows)} rows with empty text.")
    print("Here is an example of a row to be dropped:")
    
    # Display the first example
    # We use .T (transpose) to make it easier to read a single row vertically
    display(bad_rows.head(1).T) 
else:
    print("No empty text rows found in this dataset!")

Found 5 rows with empty text.
Here is an example of a row to be dropped:


,34341
id,13003338.0
title,"Interview de M. Robert Hue, secrétaire nationa..."
source_url,https://www.vie-publique.fr/discours/199811-in...
domain,Déclaration
date_pronounced,2001-10-26
speakers,"[{'speaker_name': 'Robert Hue', 'speaker_role'..."
corporate_author,[]
circumstance,None
issuer_type,Partis
document_type,Article


## Deduplication

In [22]:
# 1. Check for Exact Duplicates
# Convert lists to strings temporarily just for the check.
exact_dupes_mask = df_clean.astype(str).duplicated()
exact_dupes_count = exact_dupes_mask.sum()
print(f"Exact row duplicates found: {exact_dupes_count}")

# 2. Check for ID Duplicates (The standard way)
# This works fine because 'id' is a string/number, not a list.
id_dupes_count = df_clean.duplicated(subset=['id']).sum()
print(f"ID duplicates found: {id_dupes_count}")

# 3. Remove Duplicates
# Strategy: We prioritize the ID. If two rows have the same ID, we drop the second one.
# This avoids the 'unhashable type list' error because we are only looking at the 'id' column.
df_clean = df_clean.drop_duplicates(subset=['id'], keep='first')

print(f"Shape after deduplication: {df_clean.shape}")

Exact row duplicates found: 0
ID duplicates found: 0
Shape after deduplication: (39992, 18)


## Final Check and Wrap up

In [23]:
# Final check of the structure
display(df_clean.head(3))
df_clean.info()

# Export to JSON (Human-readable)
print("Parquet export failed (missing pyarrow?), saving to JSON.")
df_clean.to_json(file_path.replace('.json', '_clean.json'), orient='records', force_ascii=False)

,id,title,source_url,domain,date_pronounced,speakers,corporate_author,circumstance,issuer_type,document_type,type_media,media,themes,tags,date_published,date_updated,source,text
0,117000001.0,"Déclaration de M. Nicolas Sarkozy, Président d...",https://www.vie-publique.fr/discours/180848-de...,Président de la République,2010-12-31,"[{'speaker_name': 'Nicolas Sarkozy', 'speaker_...",[],"Voeux radiotélévisés aux Français, le 31 décem...",Président de la République,Déclaration,None,None,[Distinction],"[Voeux, Crise economique, Crise financière, Cr...",2019-07-05,2025-05-13,None,"Mes chers compatriotes,\nL'année 2010 s'achève..."
1,113000025.0,"Déclaration de M. Alain Juppé, ministre de la ...",https://www.vie-publique.fr/discours/180847-de...,Déclaration,2010-12-31,"[{'speaker_name': 'Alain Juppé', 'speaker_role...",[],Voeux aux personnels civils et militaires du m...,Gouvernement,Déclaration,None,None,[Politique de la défense],"[Politique de la défense, Réforme, Interventio...",2019-07-05,2025-05-13,"http://www.defense.gouv.fr, le 3 janvier 2011","En cette fin d'année 2010, quelques semaines a..."
2,112000004.0,"Communiqué de la Présidence de la République, ...",https://www.vie-publique.fr/discours/180837-co...,Communiqué,2010-12-30,[{'speaker_name': 'Présidence de la République...,[Présidence de la République],None,Présidence de la République,Communiqué,None,None,[Santé publique],"[Santé publique, Nomination, Haute autorité de...",2019-07-05,2025-05-13,None,Le Président de la République tient à saluer l...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39992 entries, 0 to 39991
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                39991 non-null  float64
 1   title             39992 non-null  object 
 2   source_url        39992 non-null  object 
 3   domain            39991 non-null  object 
 4   date_pronounced   39992 non-null  object 
 5   speakers          39992 non-null  object 
 6   corporate_author  39992 non-null  object 
 7   circumstance      22211 non-null  object 
 8   issuer_type       36570 non-null  object 
 9   document_type     39959 non-null  object 
 10  type_media        8353 non-null   object 
 11  media             9024 non-null   object 
 12  themes            39992 non-null  object 
 13  tags              39992 non-null  object 
 14  date_published    39992 non-null  object 
 15  date_updated      39992 non-null  object 
 16  source            26294 non-null  object